# Système de recommandation de produits aux clients d’un e-commerce



Charger le paquet fpgrowth_py pour les règles d'association :

pip install fpgrowth_py

# Importation de certaines bibliothèques et transformation des données

In [51]:
import numpy as np 
import pandas as pd
from fpgrowth_py import fpgrowth
import numpy as np # algèbre linéaire
import pandas as pd # data processing, entrées/sorties de fichiers CSV

# nous allons importer les bibliothèques utilisées pour l'apprentissage automatique
import numpy as np # algèbre linéaire
import pandas as pd # data processing, manipulation des données comme dans SQL
import matplotlib.pyplot as plt # utilisé pour le tracé du graphique 
import seaborn as sns # utilisé pour tracer le graphique interactif
%matplotlib inline
import time



In [52]:
data=pd.read_csv('/Users/jl/Downloads/data.csv', encoding= 'unicode_escape') # importation à partir d'un fichier csv
data['GroupPrice']=data['Quantity']*data['UnitPrice']
data=data.dropna()
print('The dimensions of the dataset are : ', data.shape)
print('---------')
data.head()

The dimensions of the dataset are :  (406829, 9)
---------


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,GroupPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [53]:
# variables explicatives :

InvoiceNo : numéro de la facture correspondant à l'achat du produit.

StockCode : Identifiant du produit acheté. Chaque identifiant est différent.

Description : Description du produit acheté.

Quantité : Quantité du produit acheté

InvoiceDate : Date de la facture, du 01/12/2010 au 09/12/2011 .

UnitPrice : Prix d'un produit.

CustomerID : Identifiant du client. Chaque identifiant est différent.

Country : Pays dans lequel le client a passé sa commande.

GroupPrice : Prix de tous les mêmes produits achetés. Quantité x Prix unitaire

# Data preprocessing

Suppression des produits qui correspondent à des cadeaux offerts par l'entreprise aux clients.
Nous ne conservons que les produits que le client a effectivement mis dans son panier.

Nous regroupons tous les produits achetés par un client.
Chaque ligne correspond à une transaction composée du numéro de facture, de l'identifiant du client et de tous les produits achetés.

In [54]:
liste= data['StockCode'].unique() 
stock_to_del=[]
for el in liste:
    if el[0] not in ['1','2','3','4','5','6','7','8','9','10']: # produits correspondant aux cadeaux 
        stock_to_del.append(el)

data=data[data['StockCode'].map(lambda x: x not in stock_to_del)] # supprimer ces produits

basket = data.groupby(['InvoiceNo','CustomerID']).agg({'StockCode': lambda s: list(set(s))}) # regrouper les produits d'une même facture 

print('Dimension of the new grouped dataset : ', basket.shape)
print('----------')
basket.head()

Dimension of the new grouped dataset :  (21788, 1)
----------


,,StockCode
InvoiceNo,CustomerID,
536365,17850.0,"[84029G, 71053, 85123A, 84406B, 84029E, 22752,..."
536366,17850.0,"[22632, 22633]"
536367,13047.0,"[22745, 22622, 84969, 21777, 21754, 22310, 226..."
536368,13047.0,"[22914, 22960, 22913, 22912]"
536369,13047.0,[21756]


# Modélisation des règles d'association : algorithme de croissance Fp

Fp Growth est un modèle de Data Mining basé sur des règles d'association.

Ce modèle permet, à partir d'un historique de transactions, de déterminer l'ensemble des règles d'association les plus fréquentes dans l'ensemble de données. Pour ce faire, il a besoin comme paramètre d'entrée de l'ensemble des transactions composées des paniers de produits que les clients ont déjà achetés.

Étant donné un ensemble de transactions, la première étape de FP-growth consiste à calculer les fréquences des articles et à identifier les articles fréquents.

La deuxième étape de FP-growth utilise une structure d'arbre suffixe (FP-tree) pour coder les transactions sans générer explicitement des ensembles candidats, qui sont généralement coûteux à générer.

Après la deuxième étape, les ensembles d'articles fréquents peuvent être extraits de l'arbre FP et le modèle renvoie un ensemble de règles d'association de produits.



Pour établir cette table, il faut fournir au modèle 2 hyperparamètres :

minSupRatio : support minimum pour qu'un itemset soit identifié comme fréquent.
Par exemple, si un élément apparaît 3 fois sur 5 transactions, son support est de 3/5=0,6.

minConf : confiance minimale pour générer une règle d'association.
La confiance est une indication de la fréquence à laquelle une règle d'association s'est avérée vraie.
Par exemple, si dans l'ensemble de transactions X apparaît 4 fois, X et Y ne coïncident que 2 fois, la confiance pour la règle X => Y est alors de 2/4 = 0,5.
Ce paramètre n'affecte pas l'extraction d'ensembles fréquents, mais spécifie la confiance minimale pour générer des règles d'association à partir d'ensembles fréquents.

Une fois les règles d'association calculées, il ne reste plus qu'à les appliquer aux paniers de produits des clients.

In [55]:
a=time.time()
freqItemSet, rules = fpgrowth(basket['StockCode'].values, minSupRatio=0.005, minConf=0.3)
b=time.time()
print('time to execute in seconds : ',b-a, ' s.')
print('Number of rules generated : ', len(rules))

association=pd.DataFrame(rules,columns =['basket','next_product','proba']) 
association=association.sort_values(by='proba',ascending=False)
print('Dimensions of the association table are : ', association.shape)
association.head(10)

time to execute in seconds :  223.52582812309265  s.
Number of rules generated :  4955
Dimensions of the association table are :  (4955, 3)


,basket,next_product,proba
197,"{22917, 22920, 22916, 22921, 22919}",{22918},0.992537
420,"{22921, 22916, 22919, 22917}",{22918},0.986014
306,"{22921, 22920, 22916, 22917}",{22918},0.985714
308,"{22921, 22920, 22918, 22917}",{22916},0.985714
92,"{22921, 22920, 22919, 22917}",{22918},0.985401
228,"{22921, 22920, 22916, 22919}",{22918},0.985401
202,"{22920, 22916, 22921, 22919, 22918}",{22917},0.985185
199,"{22917, 22920, 22921, 22919, 22918}",{22916},0.985185
364,"{22921, 22916, 22917}",{22918},0.979866
456,"{22921, 22919, 22917}",{22918},0.979730


In [56]:
def compute_next_best_product(basket_el):
    """

    paramètre : basket_el = liste des éléments du panier du consommateur

    return : next_pdt, proba = produit suivant à recommander, probabilité d'achat. Ou (0,0) si aucun produit n'est trouvé. 
            
    Description : à partir du panier d'un utilisateur, renvoie le produit à recommander s'il n'a pas été trouvé dans la liste des associations de la table associée au modèle FP Growth. 
    Pour ce faire, nous recherchons dans la table des associations le produit à recommander pour chaque produit individuel du panier du consommateur. 
    produit individuel du panier du consommateur.

    """
    
    for k in basket_el: # pour chaque élément du panier de consommation
            k={k}
            if len(association[association['basket']==k].values) !=0: # si nous trouvons une association correspondante dans la table de croissance fp
                next_pdt=list(association[association['basket']==k]['next_product'].values[0])[0] # nous prenons le produit conséquent
                if next_pdt not in basket_el : # Nous vérifions que le client n'a pas acheté le produit précédemment
                    proba=association[association['basket']==k]['proba'].values[0] # Trouver la probabilité associée
                    return(next_pdt,proba)
    
    return(0,0) # return (0,0) si aucun produit n'a été trouvé

In [57]:
def find_next_product(basket):
    """

    Paramètre : basket = dataframe du panier du consommateur

    Return : list_next_pdt, list_proba = liste des prochains éléments à recommander et les probabilités d'achat associées.
    
    description : Fonction principale qui utilise la fonction précédente.
    Pour chaque client de l'ensemble de données, nous recherchons une association correspondante dans la table du modèle Fp Growth.
    Si aucune association n'est trouvée, nous appelons la fonction compute_next_best_product qui recherche les associations de produits individuels.
    Si aucune association individuelle n'est trouvée, la fonction renvoie (0,0).

    """
    n=basket.shape[0]
    list_next_pdt=[]
    list_proba=[]
    for i in range(n): # pour chaque client
        el=set(basket['StockCode'][i]) # panier du client
        if len(association[association['basket']==el].values) !=0: # si nous trouvons une association dans la table de croissance fp correspondant à tous les paniers du client
            next_pdt=list(association[association['basket']==el]['next_product'].values[0])[0] # On prend le produit conséquent
            proba=association[association['basket']==el]['proba'].values[0] # Probabilité associée dans le tableau
            list_next_pdt.append(next_pdt)
            list_proba.append(proba)


        elif len(association[association['basket']==el].values) ==0: # Si aucun antécédent de tout le panier n'a été trouvé dans la table
            next_pdt,proba= compute_next_best_product(basket['StockCode'][i]) # fonction précédente
            list_next_pdt.append(next_pdt)
            list_proba.append(proba)
            
    return(list_next_pdt, list_proba)

# Calcul pour chaque client

In [58]:
a=time.time()
list_next_pdt, list_proba= find_next_product(basket) 
b=time.time()
print(b-a)
basket['Recommended Product']=list_next_pdt # Ensemble de produits recommandés
basket['Probability']=list_proba # Ensemble de probabilités associées
basket.head()

85.30448508262634


,,StockCode,Recommended Product,Probability
InvoiceNo,CustomerID,,,
536365,17850.0,"[84029G, 71053, 85123A, 84406B, 84029E, 22752,...",0,0.000000
536366,17850.0,"[22632, 22633]",22865,0.516393
536367,13047.0,"[22745, 22622, 84969, 21777, 21754, 22310, 226...",22750,0.593516
536368,13047.0,"[22914, 22960, 22913, 22912]",22961,0.322280
536369,13047.0,[21756],21754,0.576132


Calcul des prix estimés à partir des recommandations faites et affichage du tableau final avec l'association (client, produit recommandé)

In [59]:
basket=basket.rename(columns = {'StockCode': 'Customer basket'})
data_stock=data.drop_duplicates(subset ="StockCode", inplace = False)
prices=[]
description_list=[]
for i in range(basket.shape[0]):
    stockcode=basket['Recommended Product'][i]
    probability= basket['Probability'][i]
    if stockcode != 0:
        unitprice=data_stock[data_stock['StockCode']==stockcode]['UnitPrice'].values[0]
        description=data_stock[data_stock['StockCode']==stockcode]['Description'].values[0]
        estim_price=unitprice*probability
        prices.append(estim_price)
        description_list.append(description)
        
    else :
        prices.append(0)
        description_list.append('Null')

    

basket['Price estimation']=prices 
basket['Product description']=description_list 
basket = basket.reindex(columns=['Customer basket','Recommended Product','Product description','Probability','Price estimation'])
basket.head()

,,Customer basket,Recommended Product,Product description,Probability,Price estimation
InvoiceNo,CustomerID,,,,,
536365,17850.0,"[84029G, 71053, 85123A, 84406B, 84029E, 22752,...",0,Null,0.000000,0.000000
536366,17850.0,"[22632, 22633]",22865,HAND WARMER OWL DESIGN,0.516393,0.955328
536367,13047.0,"[22745, 22622, 84969, 21777, 21754, 22310, 226...",22750,FELTCRAFT PRINCESS LOLA DOLL,0.593516,2.225686
536368,13047.0,"[22914, 22960, 22913, 22912]",22961,JAM MAKING SET PRINTED,0.322280,0.467306
536369,13047.0,[21756],21754,HOME BUILDING BLOCK WORD,0.576132,3.427984


# Résultats



Anticipation des besoins des clients :

In [60]:
print('En moyenne, le système de recommandation peut prédire dans ',basket['Probability'].mean() *100,  '% des cas le prochain produit que le client achètera')

En moyenne, le système de recommandation peut prédire dans  36.284389697673504 % des cas le prochain produit que le client achètera


Turnover generated :

In [61]:
print('Avec un seul produit proposé, le système de recommandation peut générer un chiffre d affaires dans ce cas jusqu à : ', round(basket['Price estimation'].sum()), ' euros.') 

Avec un seul produit proposé, le système de recommandation peut générer un chiffre d affaires dans ce cas jusqu à :  20906  euros.


# Conclusion

Parmi un catalogue de produits de plus de 3000 articles, un modèle simple basé sur des règles d'association permet de prédire dans 36% des cas le prochain produit que le client achètera et donc de générer des revenus supplémentaires significatifs.

L'avantage de ce modèle est qu'il offre une très bonne précision tout en étant à la fois facile à mettre en œuvre et explicable.

En effet, contrairement à d'autres modèles d'intelligence artificielle qui peuvent apparaître comme des "boîtes noires" parce qu'ils sont difficiles à expliquer, les résultats du modèle Fp Growth sont compréhensibles parce que vous y trouverez toutes les règles spécifiques à votre entreprise.

Par exemple, si vous savez que la plupart du temps vos clients achètent le produit A et le produit B ensemble, vous le verrez immédiatement dans votre table d'association